### Here, we will demonstarte an example that tessellates the space around a given street network

### We are using networkx to analyze the street network and combine it with momepy for tessellation

In [ ]:
import osmnx as ox
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt

# Load a street network using OSMnx
place = "Jaipur, India"
G = ox.graph_from_place(place, network_type='drive')
edges = ox.graph_to_gdfs(G, nodes=False)  # Extract edges as GeoDataFrame

# Buffer the streets to create a simulated impact area
edges['geometry'] = edges.buffer(0.001)  # Adjust buffer size as needed

# Create a boundary for tessellation
boundary = edges.unary_union.convex_hull

# Generate morphological tessellation
tessellation = momepy.Tessellation(edges, unique_id='geometry', limit=boundary)

# Plot the tessellation
fig, ax = plt.subplots(figsize=(10, 10))
tessellation.tessellation.plot(ax=ax, color='lightgreen', edgecolor='black', alpha=0.5)
edges.plot(ax=ax, color='blue', label='Streets', alpha=0.7)
plt.title("Street-Based Tessellation with Momepy")
plt.legend()
plt.show()


### Enhancing Tessellation with Graph Metrics

In [ ]:
import networkx as nx

# Calculate centrality metrics
G_nx = ox.graph_from_place(place, network_type='drive')
centrality = nx.betweenness_centrality(G_nx, normalized=True)

# Add centrality as a node attribute
nx.set_node_attributes(G_nx, centrality, 'centrality')

# Visualize centrality on the street network
node_colors = [data['centrality'] for _, data in G_nx.nodes(data=True)]
ox.plot_graph(G_nx, node_color=node_colors, node_size=20, edge_linewidth=0.5, bgcolor='white')


## Combining Tessellation with Enclosures

In [ ]:
# Generate enclosures based on street network
enclosures = momepy.enclosures(edges, limit=boundary)

# Plot enclosures
fig, ax = plt.subplots(figsize=(10, 10))
enclosures.plot(ax=ax, color='lightblue', edgecolor='black', alpha=0.5)
edges.plot(ax=ax, color='blue', label='Streets', alpha=0.7)
plt.title("Enclosed Tessellation with Streets")
plt.legend()
plt.show()


## Customizing Tessellation for Urban Analysis 

In [ ]:
# Calculate area of tessellation polygons
tessellation.tessellation['area'] = tessellation.tessellation.geometry.area

# Add diversity metrics
tessellation.tessellation['diversity'] = momepy.simpson(tessellation.tessellation['area'])

# Plot tessellation with diversity
fig, ax = plt.subplots(figsize=(10, 10))
tessellation.tessellation.plot(ax=ax, column='diversity', cmap='viridis', legend=True)
edges.plot(ax=ax, color='blue', alpha=0.5)
plt.title("Tessellation with Diversity Metrics")
plt.show()
